## Installing required libraries

In [1]:
!pip install -q opencv-python
!pip install flask-ngrok


[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip


In [2]:
!pip install tabulate


[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip


In [3]:
import csv
import cv2
import itertools
import numpy as np
import pandas as pd
import os
import sys
import tempfile
import tqdm
from tqdm.autonotebook import tqdm, trange

from matplotlib import pyplot as plt
from matplotlib.collections import LineCollection

import tensorflow as tf
import tensorflow_hub as hub
from tensorflow import keras
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

ModuleNotFoundError: No module named 'tqdm'

## Pose Detection 

In [ ]:
# Download model from TF Hub and check out inference code from GitHub
!wget -q -O movenet_thunder.tflite https://tfhub.dev/google/lite-model/movenet/singlepose/thunder/tflite/float16/4?lite-format=tflite
!git clone https://github.com/tensorflow/examples.git
pose_sample_rpi_path = os.path.join(os.getcwd(), 'examples/lite/examples/pose_estimation/raspberry_pi')
sys.path.append(pose_sample_rpi_path)

# Load MoveNet Thunder model
import utils
from data import BodyPart
from ml import Movenet
movenet = Movenet('movenet_thunder')

# Define function to run pose estimation using MoveNet Thunder.
# You'll apply MoveNet's cropping algorithm and run inference multiple times on
# the input image to improve pose estimation accuracy.
def detect(input_tensor, inference_count=10):
  """Runs detection on an input image.

  Args:
    input_tensor: A [height, width, 3] Tensor of type tf.float32.
      Note that height and width can be anything since the image will be
      immediately resized according to the needs of the model within this
      function.
    inference_count: Number of times the model should run repeatly on the
      same input image to improve detection accuracy.

  Returns:
    A Person entity detected by the MoveNet.SinglePose.
  """
  image_height, image_width, channel = input_tensor.shape

  # Detect pose using the full input image
  movenet.detect(input_tensor.numpy(), reset_crop_region=True)

  # Repeatedly using previous detection result to identify the region of
  # interest and only croping that region to improve detection accuracy
  for _ in range(inference_count - 1):
    person = movenet.detect(input_tensor.numpy(),
                            reset_crop_region=False)

  return person

Cloning into 'examples'...
remote: Enumerating objects: 23460, done.
remote: Counting objects: 100% (420/420), done.
remote: Compressing objects: 100% (266/266), done.
remote: Total 23460 (delta 121), reused 356 (delta 86), pack-reused 23040
Receiving objects: 100% (23460/23460), 43.94 MiB | 23.99 MiB/s, done.
Resolving deltas: 100% (12778/12778), done.


In [ ]:

def draw_prediction_on_image(
    image, person, crop_region=None, close_figure=True,
    keep_input_size=False):
  """Draws the keypoint predictions on image.

  Args:
    image: An numpy array with shape [height, width, channel] representing the
      pixel values of the input image.
    person: A person entity returned from the MoveNet.SinglePose model.
    close_figure: Whether to close the plt figure after the function returns.
    keep_input_size: Whether to keep the size of the input image.

  Returns:
    An numpy array with shape [out_height, out_width, channel] representing the
    image overlaid with keypoint predictions.
  """
  # Draw the detection result on top of the image.
  image_np = utils.visualize(image, [person])

  # Plot the image with detection results.
  height, width, channel = image.shape
  aspect_ratio = float(width) / height
  fig, ax = plt.subplots(figsize=(12 * aspect_ratio, 12))
  im = ax.imshow(image_np)

  if close_figure:
    plt.close(fig)

  if not keep_input_size:
    image_np = utils.keep_aspect_ratio_resizer(image_np, (512, 512))

  return image_np

In [ ]:

class MoveNetPreprocessor(object):
  """Helper class to preprocess pose sample images for classification."""

  def __init__(self,
               images_in_folder,
               images_out_folder,
               csvs_out_path):
    #Creates a preprocessor to detection pose from images and save as CSV.
    self._images_in_folder = images_in_folder
    self._images_out_folder = images_out_folder
    self._csvs_out_path = csvs_out_path
    self._messages = []

    # Create a temp dir to store the pose CSVs per class
    self._csvs_out_folder_per_class = tempfile.mkdtemp()

    # Get list of pose classes and print image statistics
    self._pose_class_names = sorted(
        [n for n in os.listdir(self._images_in_folder) if not n.startswith('.')]
        )

  def process(self, per_pose_class_limit=None, detection_threshold=0.1):
    #Preprocesses images in the given folder.

    # Loop through the classes and preprocess its images
    for pose_class_name in self._pose_class_names:
      print('Preprocessing', pose_class_name, file=sys.stderr)

      # Paths for the pose class.
      images_in_folder = os.path.join(self._images_in_folder, pose_class_name)
      images_out_folder = os.path.join(self._images_out_folder, pose_class_name)
      csv_out_path = os.path.join(self._csvs_out_folder_per_class,
                                  pose_class_name + '.csv')
      if not os.path.exists(images_out_folder):
        os.makedirs(images_out_folder)

      # Detect landmarks in each image and write it to a CSV file
      with open(csv_out_path, 'w') as csv_out_file:
        csv_out_writer = csv.writer(csv_out_file,
                                    delimiter=',',
                                    quoting=csv.QUOTE_MINIMAL)
        # Get list of images
        image_names = sorted(
            [n for n in os.listdir(images_in_folder) if not n.startswith('.')])
        if per_pose_class_limit is not None:
          image_names = image_names[:per_pose_class_limit]

        valid_image_count = 0

        # Detect pose landmarks from each image
        for image_name in tqdm.tqdm(image_names):
          image_path = os.path.join(images_in_folder, image_name)

          try:
            image = tf.io.read_file(image_path)
            image = tf.io.decode_jpeg(image)
          except:
            self._messages.append('Skipped ' + image_path + '. Invalid image.')
            continue
          else:
            image = tf.io.read_file(image_path)
            image = tf.io.decode_jpeg(image)
            image_height, image_width, channel = image.shape

          # Skip images that isn't RGB because Movenet requires RGB images
          if channel != 3:
            self._messages.append('Skipped ' + image_path +
                                  '. Image isn\'t in RGB format.')
            continue
          person = detect(image)

          # Save landmarks if all landmarks were detected
          min_landmark_score = min(
              [keypoint.score for keypoint in person.keypoints])
          should_keep_image = min_landmark_score >= detection_threshold
          if not should_keep_image:
            self._messages.append('Skipped ' + image_path +
                                  '. No pose was confidentlly detected.')
            continue

          valid_image_count += 1

          # Draw the prediction result on top of the image for debugging later
          output_overlay = draw_prediction_on_image(
              image.numpy().astype(np.uint8), person,
              close_figure=True, keep_input_size=True)

          # Write detection result into an image file
          output_frame = cv2.cvtColor(output_overlay, cv2.COLOR_RGB2BGR)
          cv2.imwrite(os.path.join(images_out_folder, image_name), output_frame)

          # Get landmarks and scale it to the same size as the input image
          pose_landmarks = np.array(
              [[keypoint.coordinate.x, keypoint.coordinate.y, keypoint.score]
                for keypoint in person.keypoints],
              dtype=np.float32)

          # Write the landmark coordinates to its per-class CSV file
          coordinates = pose_landmarks.flatten().astype(np.str).tolist()
          csv_out_writer.writerow([image_name] + coordinates)

        if not valid_image_count:
          raise RuntimeError(
              'No valid images found for the "{}" class.'
              .format(pose_class_name))

    # Print the error message collected during preprocessing.
    print('\n'.join(self._messages))

    # Combine all per-class CSVs into a single output file
    all_landmarks_df = self._all_landmarks_as_dataframe()
    all_landmarks_df.to_csv(self._csvs_out_path, index=False)

  def class_names(self):
    #List of classes found in the training dataset."""
    return self._pose_class_names

  def _all_landmarks_as_dataframe(self):
    #"""Merge all per-class CSVs into a single dataframe."""
    total_df = None
    for class_index, class_name in enumerate(self._pose_class_names):
      csv_out_path = os.path.join(self._csvs_out_folder_per_class,
                                  class_name + '.csv')
      per_class_df = pd.read_csv(csv_out_path, header=None)

      # Add the labels
      per_class_df['class_no'] = [class_index]*len(per_class_df)
      per_class_df['class_name'] = [class_name]*len(per_class_df)

      # Append the folder name to the filename column (first column)
      per_class_df[per_class_df.columns[0]] = (os.path.join(class_name, '')
        + per_class_df[per_class_df.columns[0]].astype(str))

      if total_df is None:
        # For the first class, assign its data to the total dataframe
        total_df = per_class_df
      else:
        # Concatenate each class's data into the total dataframe
        total_df = pd.concat([total_df, per_class_df], axis=0)

    list_name = [[bodypart.name + '_x', bodypart.name + '_y',
                  bodypart.name + '_score'] for bodypart in BodyPart]
    header_name = []
    for columns_name in list_name:
      header_name += columns_name
    header_name = ['file_name'] + header_name
    header_map = {total_df.columns[i]: header_name[i]
                  for i in range(len(header_name))}

    total_df.rename(header_map, axis=1, inplace=True)

    return total_df

In [ ]:
import os
import difflib
import pandas as pd
df = pd.DataFrame()
df3 = pd.DataFrame()

def string_similarity(str1, str2):
    result =  difflib.SequenceMatcher(a=str1.lower(), b=str2.lower())
    return result.ratio()
def calculation(heights,imgroute, output_overlayed = False):
  z = imgroute
  image = tf.io.read_file(z)
  height = heights
  image = tf.io.decode_jpeg(image)
  pheight = image.get_shape()[0]
  person = detect(image)
  keys = []
  #Y-axis Distortion Correction
  def ydistortionWrapper(cheight, cdistance, sheight, numpixels):
    def ydistortion(pycoord):
      a1 =np.arctan2(cheight,cdistance)
      a2= np.arctan2((sheight-cheight),cdistance)
      p2a = numpixels/(a1+a2)
      a3 = (pheight-pycoord)/p2a

      return cdistance*(np.tan(a1)-np.tan(a1-a3))
    return ydistortion
  #X-axis Distortion Correction
  def xdistortionWrapper(cheight, cdistance, sheight, numpixels):
    def xdistortion(pxcoord):
      a1 =np.arctan2(cheight,cdistance)
      a2= np.arctan2((sheight-cheight),cdistance)
      p2a = numpixels/(a1+a2)
      a3 = pxcoord/p2a
      return pxcoord*cdistance/p2a
    return xdistortion
  #Distance Calculation
  def distance1(x1, y1, x2, y2):
    ydis = ydistortionWrapper(62,111,heights,pheight)
    xdis = xdistortionWrapper(62,111,heights,pheight)
    v = np.sqrt(pow(xdis(x1)-xdis(x2),2)+pow(ydis(y1)-ydis(y2),2))
    return v
  #Index the points to an array
  while(len(keys)==0):
    for z1 in range(len(person.keypoints)):
      if(z1 in [5,6,7,8,9,10,11,12,13,14,15,16]):
        keys.append(person.keypoints[z1])

  d1 = distance1(keys[6].coordinate.x,keys[6].coordinate.y,keys[8].coordinate.x,keys[8].coordinate.y)

  d2 = distance1(keys[7].coordinate.x,keys[7].coordinate.y,keys[9].coordinate.x,keys[9].coordinate.y)

  e1 = distance1(keys[1].coordinate.x,keys[1].coordinate.y,keys[3].coordinate.x,keys[3].coordinate.y)+distance1(keys[3].coordinate.x,keys[3].coordinate.y,keys[5].coordinate.x,keys[5].coordinate.y)
  e2 = distance1(keys[0].coordinate.x,keys[0].coordinate.y,keys[2].coordinate.x,keys[2].coordinate.y)+distance1(keys[2].coordinate.x,keys[2].coordinate.y,keys[4].coordinate.x,keys[4].coordinate.y)

  f1 = distance1(keys[0].coordinate.x,keys[0].coordinate.y,keys[1].coordinate.x,keys[1].coordinate.y)
  b1 = distance1(0,pheight,0,keys[0].coordinate.y)
  b2 = distance1(0,pheight,0,keys[1].coordinate.y)
  c2 = distance1(keys[9].coordinate.x,keys[9].coordinate.y,keys[11].coordinate.x,keys[11].coordinate.y)+d2
  c1 = distance1(keys[8].coordinate.x,keys[8].coordinate.y,keys[10].coordinate.x,keys[10].coordinate.y)+d1

  pred = [(((b1+b2)/2)),(((c1+c2)/2)),(((d1+d2)/2)),(((e1+e2)/2)),(f1)]
  #Return prediction or (prediction, overlayed image) for use in analyze_and_display
  if output_overlayed:
    overlayed = utils.visualize(image.numpy(), [person])
    return (pred, overlayed)
  
  return pred


In [ ]:
from __future__ import print_function
import pickle

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

import sqlite3 as lite

import matplotlib.pyplot as plt
## height of user and file path for image
def analyze(height, imgroute):

  with open('/content/drive/MyDrive/Bike Pose Detection Folder/file.pkl', 'rb') as fid:
     reg = pickle.load(fid)
     ogpredict = [calculation(height,imgroute)]
     prediction=reg.predict(ogpredict)
     final_prediction=[height,prediction[0][0],prediction[0][1],prediction[0][2],prediction[0][3]]
     return final_prediction



## New functions for integration and angle finding


In [ ]:
#decompose the predictions
def decompose_to_dictionary(prediction_array):
  """
  Input: Array format [B: shoulder height, C: Inseam, D: Thigh, E:Arm length, F: Eye to shoulder]
  DOES NOT MODIFY INPUT
  Output: Returns dictionary "dimension name": Value in inches
  """
  base = {
      "height": prediction_array[0],
     "sh_height": prediction_array[1],
     "inseam": (prediction_array[2]),
     "up_leg": prediction_array[3],
     "arm_len": prediction_array[4],
     #"sh_width": prediction_array[5],
  }
  base["tor_len"] = base["sh_height"] - base["inseam"]
  base["low_leg"] = base["inseam"] - base["up_leg"]
  #base["up_leg"] += 4

  return base

In [ ]:
import numpy as np
from scipy.stats import norm
from tabulate import tabulate

# #####
# Bike Ergonomic Angle Fit Calculator
# Input: Vector of bike coordinates (seat and handle bar position) bottom bracket is origin
# Input: Vecor of body dimensions (shoulder height, leg length, upper leg length, arm length)
# Input: use case ie (road, mtb, commuter/comfort)
# Output: Probabilities of deviation from optimal angles
######


# dict of dictionaries maping use cases to their respective body angles and variations
USE_DICT = {
    "road": {
        "opt_knee_angle": (37.5, 5),
        "opt_back_angle": (45, 5),
        "opt_awrist_angle": (90, 5),
        "opt_elbow_angle": (160, 10),
        "opt_ankle_angle": (100.0, 5.0),
        "opt_hip_angle_closed": (60, 5),
    },
    "mtb": {
        "opt_knee_angle": (37.5, 2.5),
        "opt_back_angle": (45, 5),
        "opt_elbow_angle": (160, 10),
        "opt_ankle_angle": (100.0, 5.0),
        "opt_hip_angle_closed": (60, 5),
    },
    "tt": {
        "opt_knee_angle": (37.5, 2.5),
        "opt_back_angle": (45, 5),
        "opt_elbow_angle": (160, 10),
        "opt_ankle_angle": (100.0, 5.0),
        "opt_hip_angle_closed": (60, 5),
    },
    "commute": {
        "opt_knee_angle": (37.5, 2.5),
        "opt_back_angle": (45, 5),
        "opt_elbow_angle": (160, 10),
        "opt_ankle_angle": (100.0, 5.0),
        "opt_hip_angle_closed": (60, 5),
    },
}

###################################
# FUNCTIONS FOR CALCUATING ANGLES #
###################################
def knee_extension_angle(bike_vector, body_vector, CA, tor=False):
    """
    Input:
        bike vector, body vector, crank angle
        np array bike vector:
            [SX, SY, HX, HY, CL]^T
            (seat_x, seat_y, hbar_x, hbar_y, crank len)
        np array body vector:
            [LL, UL, TL, AL, FL, AA]
            (lowleg, upleg, torso len, arm len, foot len, ankl angle)
        CA = crank angle:
            crank angle fom horizontal in radians
        Origin is bottom bracket

    Output:
        Knee extension angle
        OR
        None if not valid coords (i.e. NaA appeared)

    """
    #decomposing body vector
    sq_body = np.square(body_vector)

    LL = body_vector[0, 0]
    UL = body_vector[1, 0]
    TL = body_vector[2, 0]
    AL = body_vector[3, 0]
    FL = body_vector[4, 0]
    AA = body_vector[5, 0]

    LL_s = sq_body[0, 0]
    UL_s = sq_body[1, 0]
    TL_s = sq_body[2, 0]
    AL_s = sq_body[3, 0]
    FL_s = sq_body[4, 0]
    AA_s = sq_body[5, 0]

    # decomposing bike vector
    sq_bike = np.square(bike_vector)

    SX = bike_vector[0, 0]
    SY = bike_vector[1, 0]
    HX = bike_vector[2, 0]
    HY = bike_vector[3, 0]
    CL = bike_vector[4, 0]

    SX_s = sq_bike[0, 0]
    SY_s = sq_bike[1, 0]
    HX_s = sq_bike[2, 0]
    HY_s = sq_bike[3, 0]
    CL_s = sq_bike[4, 0]


    #using law of sines and checks for nan/angle validity
    x_1 = np.sqrt(LL_s + FL_s - (2 * LL * FL * np.cos(AA)))

    LX = CL * np.cos(CA) - SX
    LY = SY - CL * np.sin(CA)

    x_2 = np.sqrt((LX**2 + LY**2))

    alpha_1 = np.arccos((x_1**2 - UL_s - x_2**2) / (-2 * UL * x_2))
    if np.isnan(alpha_1):
        return None

    alpha_2 = np.arctan2(LY, LX) - alpha_1

    LLY = LY - UL * np.sin(alpha_2)
    LLX = LX - UL * np.cos(alpha_2)

    alpha_3 = np.arctan2(LLY, LLX) - alpha_2

    alpha_4 = np.arccos((FL_s - LL_s - x_1**2) / (-2 * LL * x_1))
    if np.isnan(alpha_4):
        return None


    return alpha_3 + alpha_4


def back_armpit_angles(bike_vector, body_vector, elbow_angle):
    """
    Input: bike_vector, body_vector, elbow_angle
    Output: back angle, armpit to elbow angle, armpit to wrist angle in degrees

    np array bike vector:
            [SX, SY, HX, HY, CL]^T
    np array body vector:
            [LL, UL, TL, AL, FL, AA]
    """
    elbow_angle = elbow_angle * (np.pi/180)

    LL = body_vector[0, 0]
    UL = body_vector[1, 0]
    TL = body_vector[2, 0]
    AL = float(body_vector[3, 0])
    FL = body_vector[4, 0]
    AA = float(body_vector[5, 0])

    # decomposing bike vector
    SX = bike_vector[0, 0]
    SY = bike_vector[1, 0]
    HX = bike_vector[2, 0]
    HY = bike_vector[3, 0]
    CL = bike_vector[4, 0]

    #### BACK ANGLE ####
    # Calculating straightline distance between handlebars and seat
    sth_dist = ((HY - SY) ** 2 + (HX - SX) ** 2) ** 0.5

    # Calculating angle offset (horizontal to sth_dist) for torso angle
    sth_ang = np.arctan2((HY - SY), (HX - SX))


    # Uses new dist and law of cosines to find torso angle
    x_1 = (AL / 2) ** 2 + (AL / 2) ** 2 - 2 * (AL / 2) * (AL / 2) * np.cos(elbow_angle)
    tors_ang = np.arccos((TL**2 + sth_dist**2 - x_1) / (2 * TL * sth_dist))
    #if not possible return None
    if np.isnan(tors_ang):
        return (None, None)

    # Adds offset to get back angle with horizontal
    back_angle = tors_ang + sth_ang

    #### ARMPIT TO WRIST ANGLE ####
    armpit_to_wrist = np.arccos((TL**2 + x_1 - sth_dist**2)/ (2 * TL * (x_1**0.5)))

    # return angles in radians
    return (back_angle, armpit_to_wrist)


def deg_to_r(deg):
    """ 
    Converts degrees to radians
    """
    return float(deg / 180) * np.pi


def rad_to_d(rad):
    """
    Converts radians to degrees
    """
    return float(rad * (180 / np.pi))

def all_angles(bike_vector, body_vector, arm_angle):
  """
  Input: bike, body, arm_angle in degrees
  Output: tuple (min_ke angle, back angle, awrist angle) in degrees
  """

  # min knee extension angle over sweep 0-2pi
  ke_ang = [
            item for item in (knee_extension_angle(bike_vector, body_vector, angle*0.2)
            for angle in range(0, 32)) if item != None
        ]
  if ke_ang == [] or len(ke_ang) != 32:
    ke_ang = None
  else:
    ke_ang = min(ke_ang)


# back angle, armpit to elbow angle, armpit to wrist angle
  b_ang, aw_ang = back_armpit_angles(bike_vector, body_vector, arm_angle)

  return [rad_to_d(ang) if ang != None else None for ang in [ke_ang, b_ang, aw_ang]]

############################
# FUNCTIONS FOR PROABILITY #
############################

def prob(mean, sd, value):
    """
    Returns probability of value or larger given mean and sd
    """
    #handle None type values
    if value is None:
      return None

    dist = abs((value - mean))/sd
    return 1 - norm.cdf(dist, loc=0, scale=1)

def prob_dists(bike_vector, body_vector, arm_angle, use="road"):
    """
    Input: bike, body, arm angle (degrees), [optional] usecase
    Output: Computes probability of deviation from optimal for each body angle
    """
    # back angle, armpit to elbow angle, armpit to wrist angle
    our_knee_angle, our_back_angle, our_awrist_angle = all_angles(bike_vector, body_vector, arm_angle)

    k_ang_prob = prob(
        USE_DICT[use]["opt_knee_angle"][0],
        USE_DICT[use]["opt_knee_angle"][1],
        our_knee_angle,
    )
    b_ang_prob = prob(
        USE_DICT[use]["opt_back_angle"][0],
        USE_DICT[use]["opt_back_angle"][1],
        our_back_angle,
    )
    aw_ang_prob = prob(
        USE_DICT[use]["opt_awrist_angle"][0],
        USE_DICT[use]["opt_awrist_angle"][1],
        our_awrist_angle,
    )

    return (k_ang_prob, b_ang_prob, aw_ang_prob)


In [ ]:
#######################
# FUNCTIONS FOR NOISE #
#######################
def all_noise(bike, body, step_size, n):
  """
  Input: bike, body, step size, n output points
  Output: prints tables corresponding to dimension noise changes
  for all bike and body dimensions
  """

  def test_noise(dim_type, dimension_i):
    """
    Input: dimension type ("bike" or "body"), dimension index to test
    Output: Returns list of 2n-1 tuples: (new_dim value, new_t_angels values, noise amt, t_angles changes)
    """
    out = []
    step = step_size
    new_bike = bike[:,:]
    new_body = body[:,:]
    o_ke, o_ba, o_aw = all_angles(new_bike, new_body, 150)

    #Generate line for table that handles none values
    def line_gen(dim, noise_amt, ke, back, awrist):
      if ke is None:
        ke_dif = "NaN"
        new_ke = "NaN"
      else:
        ke_dif = ke-o_ke
        new_ke = ke

      if back is None:
        ba_dif = "NaN"
        new_back = "NaN"
      else:
        ba_dif = back-o_ba
        new_back = back

      if awrist is None:
        aw_dif = "NaN"
        new_awrist = "NaN"
      else:
        aw_dif = awrist-o_aw
        new_awrist = awrist

      return (dim, noise_amt, ke_dif, ba_dif, aw_dif, new_ke, new_back, new_awrist)

    #makes 2n - 1 Data Points
    for i in range(0, n):
      noise_amt = i * step

      if dim_type == "bike":
        #positive noise
        new_bike[dimension_i, 0] += noise_amt
        ke, back, awrist = all_angles(new_bike, new_body, 150)
        out.append(line_gen(new_bike[dimension_i, 0], noise_amt, ke, back, awrist))

        #negative noise
        new_bike[dimension_i, 0] -= (2*noise_amt)
        ke, back, awrist = all_angles(new_bike, new_body, 150)
        if i != 0:
          out.append(line_gen(new_bike[dimension_i, 0], -noise_amt, ke, back, awrist))

        #reset bike
        new_bike[dimension_i, 0] += noise_amt


      elif dim_type == "body":
        #positive noise
        new_body[dimension_i, 0] += noise_amt
        ke, back, awrist = all_angles(new_bike, new_body, 150)
        out.append(line_gen(new_body[dimension_i, 0], noise_amt, ke, back, awrist))


        #negative noise
        new_body[dimension_i, 0] -= (2*noise_amt)
        ke, back, awrist = all_angles(new_bike, new_body, 150)
        if i != 0:
          out.append(line_gen(new_body[dimension_i, 0], -noise_amt, ke, back, awrist))

        #reset body
        new_body[dimension_i, 0] += noise_amt

      else:
        raise ValueError("dim_type must be either 'bike' or 'body'", dim_type)

    out.sort(key = lambda x: x[1])
    print(tabulate(out, headers=["Dim Value", "Noise Amt", "ke dif", "back dif", "awrist diff", "ke", "back", "awrist"]))

  #dict for dimension
  bike_dims = {
      0: "Seat X",
      1: "Seat Y",
      2: "Handlebar X",
      3: "Handlebar Y",
  }
  body_dims = {
      0: "Lower Leg",
      1: "Upper Leg",
      2: "Torso Length",
      3: "Arm Length",
      4: "Foot Length",
      5: "Ankle Angle",
  }
  #bike noise
  print("#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*\n#*#*#*#*# BIKE NOISE #*#*#*#*#\n#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*")

  for key, name in bike_dims.items():
    print(f"\n ***** {name} Dimension *****")
    test_noise("bike", key)


  #body noise
  print("\n\n\n#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*\n#*#*#*#*# BODY NOISE #*#*#*#*#\n#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*")
  for key, name in body_dims.items():
    print(f"\n ***** {name} Dimension *****")
    test_noise("body", key)








In [2]:
#To print table from output of analyze_folder
def print_analyze_table(out):
    """
    Input: Output of analyze_folder
    Output: Prints table of results
    """
    #new out folder to discard numpy images
    new_out = [line[:-1] for line in out]
    print(tabulate(new_out, headers = ["file", "pred torso", "pred upleg", "pred lowleg", "pred arm", "dtorso", "dupleg", "dlowleg", "darm", "avgdif"]))

In [1]:
def display_images(input_list):
  """Displays images in input list: [image, title, differences]"""
  rows = int(len(input_list)/2) + 1
  columns = 2
  plt.rcParams["figure.figsize"] = [16, 6*rows]
  plt.rcParams["figure.autolayout"] = True

  for i, tup in enumerate(input_list):
    #adds overlay and title to img plot
    plt.subplot(rows, columns, i+1)
    plt.axis("off")
    plt.imshow(tup[0])
    plt.title(tup[1])
    plt.xlabel(tup[2])
  
  plt.show()

In [ ]:
#To display images from output of analyze_folder
def display_analyze_images(out):
    """
    Input: Output of analyze_folder
    Output: Displays images of results
    """
    new_out = []
    for line in out:
        img = line[-1]
        title = line[0]
        x_label = (f"dtorso: {line[5]}, dupleg: {line[6]}, dlowleg: {line[7]}, darm: {line[8]}, avgdif: {line[9]}")
        new_out.append((img, title, x_label))
    display_images(new_out)


In [ ]:
# Analyze all pictures in folder; name format: "[name]-[identifier]-[cam dist]-[cam height].jpg" can end in "".JPG" also
import glob
def analyze_folder(folder, users, display_images = False):
  """
  Input:
      1. Folder of pics
            IMPORTANT: name format: "[name]-[identifier]-[cam dist]-[cam height].jpg"
      2. User dimensions dictionary: {"name": {"height": height... "torso", "upleg", "lowleg", "arm"}}
  Output: 
    Prints table: Picure Name: Predicted Dimensions | Difference From Actual Dimensions
    Returns: List of tuples corresponding to rows of table + POSE IMAGE AS NP ARRAY
      (file name, pred torso, pred upleg, pred lowleg, pred arm, dtorso, dupleg, dlowleg, darm, image)
  """
  #iterator for all pictures (end in .jpg) in folder path
  paths = glob.glob((folder + "/*.jpg")) + glob.glob((folder + "/*.JPG"))

  out = []
  for i, pic in enumerate(paths):
    #break up file name
    file_name = pic.split("/")[-1][:-4]
    file_name = file_name.split("-")
    
    #skip file if doesn't match format
    if len(file_name) != 4:
        continue
    
    name, identifier, camx, camy = file_name
    print(f"Processing: {i}/{len(paths)}: {pic}")

    if name in users:
      #run model on file
      result, overlayed = analyze(users[name]["height"], pic, output_overlayed = True)
      pred = decompose_to_dictionary(result)

      #calculate difference pred - real value
      dtorso = pred["tor_len"] - users[name]["torso"]
      dupleg = pred["up_leg"] - users[name]["upleg"]
      dlowleg = pred["low_leg"] - users[name]["lowleg"]
      darm = pred["arm_len"] - users[name]["arm"]
      davg = (abs(dtorso) + abs(dupleg) + abs(dlowleg) + abs(darm))/4
      
      #store to output list
      line = ((name + "-" + identifier), pred["tor_len"], pred["up_leg"], pred["low_leg"], pred["arm_len"], dtorso, dupleg, dlowleg, darm, davg, overlayed)
      out.append(line)

  #print table
  print_analyze_table(out)
  
  #display images if desired
  if display_images:
    display_analyze_images(out)
     
  return out


In [ ]:
def display_poses(folder):
  """
  Input: Takes folder path
  Output: Displays images with joint predictions overlayed
  """
  paths = glob.glob((folder + "/*.jpg")) + glob.glob((folder + "/*.JPG"))
  
  #gets all image, person tuples to be processed later
  out = []
  for i, pic in enumerate(paths):
    #break up file name
    file_name = pic.split("/")[-1][:-4]
    file_name = file_name.split("-")
    
    #skip file if doesn't match format
    if len(file_name) != 4:
        continue

    #makes image id name
    name, identifier, camx, camy = file_name
    pic_id = (name + "-" + identifier)
    
    #preprocess for overlaying
    image = tf.io.read_file(pic)
    image = tf.io.decode_jpeg(image)
    person = detect(image)


    #make new img with overlayed pose
    overlayed = utils.visualize(image.numpy(), [person])
    
    #add to out list
    out.append((overlayed, pic_id))
  

  #processes out list into plt figure
  #new figure with 2 cols and enough rows for all of out list
  display_images(out)
  return out

## Tools Section 

In [ ]:
#Allows for use on google colaboratory
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Fill in the body dimension and bike vector to get the bike fit data

In [ ]:
#Tests for all angles and probability
LL = 19
UL = 15.5
TL = 21
AL = 24
FL = 5.5
AA = deg_to_r(107)
body_4 = np.array([[LL, UL, TL, AL, FL, AA]]).T
bike_4 = np.array([[-9., 27, 16.5, 25.25, 7]]).T

print(all_angles(bike_4, body_4, 150))

print("###### Probability ######")
prob_out = prob_dists(bike_4, body_4, 150)
print(prob_out)

Fill in bike and body vectors, modify step size, and determine + data points for noise analysis.

In [ ]:
#TESTING NOISE GENERATION
body_4 = np.array([[LL, UL, TL, AL, FL, AA]]).T
bike_4 = np.array([[-9., 27, 16.5, 25.5, 7.]]).T

all_noise(bike_4, body_4, 0.25, 4)

Define and fill our_users dictionary, give valid path to photos, and run to get prediction difference table.

Ensure photos are in correct format ([name]-[identifier]-[cam dist]-[cam height].jpg) and are .jpg or .JPG files.

In [ ]:
#Testing Folder Analysis
our_users = {}
noah_dict = {"height": 71, "torso": 21., "upleg": 15.5, "lowleg": 19., "arm": 24.}
faez_dict = {"height": 70, "torso": 28.3, "upleg": 15.5, "lowleg": 19., "arm": 21.25}
our_users["noah"]= noah_dict
our_users["faez"] = faez_dict

folder = "/Users/noahwiley/Library/Mobile Documents/com~apple~CloudDocs/Documents/MIT/Bike UROP/MeasureML-main/pictures/"
results = analyze_folder(folder, our_users)

Use this to sort the prediction table by a given column. Fill in header column name and run.

In [ ]:
header_name ={
    "file": 0, "pred torso": 1, "pred upleg": 2, "pred lowleg": 3, "pred arm": 4, "dtorso": 5, "dupleg": 6, "dlowleg": 7, "darm": 8, "avgdif": 9         
}
#modify header_name to column of interest
results.sort(key= lambda x: x[header_name["avgdif"]])

print(tabulate(results, headers = ["file", "pred torso", "pred upleg", "pred lowleg", "pred arm", "dtorso", "dupleg", "dlowleg", "darm", "avgdif"]))